In [7]:
import torch
import tensorflow as tf
import os
from PIL import Image
import torchvision
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader,random_split
from torchvision import transforms
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim

In [8]:
class Custom_data(Dataset):
    def __init__(self,root_dir,transform=None):
        self.root_dir=root_dir
        self.transform=transform
        self.file_paths=[]
        self.class_names=sorted(os.listdir(root_dir))

        for class_name in self.class_names:
            class_path=os.path.join(self.root_dir,class_name)
            files=[os.path.join(class_path, file) for file in os.listdir(class_path)]
            self.file_paths.extend(files)

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self,idx):
        img_path=self.file_paths[idx]
        image=Image.open(img_path)

        if self.transform:
            image=self.transform(image)

        class_name=os.path.basename(os.path.dirname(img_path))
        label_idx=torch.tensor(self.class_names.index(class_name),dtype=torch.long)
        return image,label_idx

In [9]:
transform=transforms.Compose([transforms.Resize((256,256)),transforms.ToTensor()])
dataset=Custom_data(root_dir="D:\\SEM-6\\DL-LAB\\Medicinal_plants_DS",transform=transform)

In [10]:
print("Length of dataset:",len(dataset))
train_size=int(0.8*len(dataset))
test_size=len(dataset)-train_size
train_dataset,test_dataset=random_split(dataset,[train_size,test_size])

Length of dataset: 2434


In [11]:
train_loader=DataLoader(train_dataset,batch_size=64,shuffle=True)
val_loader=DataLoader(test_dataset,batch_size=64,shuffle=False)

In [12]:
class neuralNet(nn.Module):
    def __init__(self,inp,out):
        super(neuralNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(inp, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, out)
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [13]:
epochs=5
batch_size=64
lr=0.1
inp_size=256*256*3
out_size=29

writer=SummaryWriter(log_dir='./log2')
ml=neuralNet(inp_size,out_size)

images, labels = next(iter(train_loader))

grid = torchvision.utils.make_grid(images)
writer.add_image('images', grid, 0)
writer.add_graph(ml, images)


optimizer=optim.Adam(ml.parameters(),lr=lr)
criterion = nn.CrossEntropyLoss()


In [14]:
for epoch in range(epochs):
    total_loss=0
    for images,labels in train_loader:
        outputs=ml(images)
        loss=criterion(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
    avg_loss=total_loss/len(train_loader)

    writer.add_scalar('Loss',avg_loss,global_step=epoch)
    

    print("\nEpoch",epoch+1,"Loss:",avg_loss)

        


Epoch 1 Loss: 5785.166133488378

Epoch 2 Loss: 3.0576322770887807

Epoch 3 Loss: 2.988828412948116

Epoch 4 Loss: 2.977351588587607

Epoch 5 Loss: 2.9787642648143153


In [15]:
writer.close()